![figure](lab3/politecnico_h-01.png)

# **Programmable Electronics**
#### Bachelor's Degree in Electrical and Computer Engineering 

## Laboratory Work 3 - Sequential Circuits 

-----------------------


## Introduction ##
In this work we will implement a set of sequential electronic circuits using a behavioral description in VHDL, which allows the implementation of medium-scale circuits using a higher level of abstraction. This document does not provide details on how to use Vivado. Whenever you have questions at this level, consult the Lab1 statement.


### Objectives ###

In this work we intend to observe how the behavioral specification can be used to implement different types of medium-scale sequential circuits, such as registers, memories, counters. To this end, we propose the implementation of 3 sequential circuits:
* Temporal multiplexing system for 7-segment displays.
* FIFO (Fist-In-First-Out) registration.
* Digital stopwatch.


In the instructions below **{sources}** refers to `C:\Xilinx\EEC_EP\sources` and **{labs}** referes to `(C:\Xilinx\EEC_EP\labs)`


_______________________

## Step 1 - Temporal multiplexing system for 7-segment displays  ##

### Step 1.1 ### 

Interfacing to multi-element 7-segment displays is typically done by scanning, to reduce FPGA pin usage. This means that the inputs to the display array are time multiplexed. Each display has an individual enable input ( $an_i$
 , i=1...4) but shares its data inputs with all other displays. Note that the enable input is asserted to zero, as shown in the figure. As long as the refresh rate is high enough, the human eye cannot tell that the displays are scan-driven.


<div>
<img src="lab3/fig1.png" width="800"/>
</div>

Temporal multiplexing can be performed before or after the hexadecimal-to-7-segment conversion (see the figure below). Since binary representation is more common in most applications, we will assume that the conversion to 7-segment is performed after the multiplexing circuit. This scheme requires a hexadecimal-to-7-segment decoder, a 4-bit vector multiplexer, a counter, and a zero-asserted 2:4 decoder. The VHDL code corresponding to the circuit shown in the figure is provided in the file **disp_hex_mux.vhd**.


<div>
<img src="lab3/fig2.png" width="800"/>
</div>


The refresh rate must be greater than 100Hz to ensure that the scan is not noticeable to the human eye. The development board is equipped with a  $f_{osc}$ =100MHz, so we need a clock divider. The simplest circuit to implement a clock divider is a counter. The LSB (bit 0) of a counter oscillates at frequency $f_{osc}$ / $2$. By analogy, bit n will oscillate at a frequency  $f_{osc}$ / $2^{n+1}$. Therefore, bit n=17 of a binary counter oscillates at a frequency of 100MHz / $2^{18}$ = 381Hz.


* Start by downloading the provided source files (Github or Moodle) and saving them in  `{sources}/lab3`.


* Open Vivado from the icon on your desktop or: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Select **Create Project** to start the wizard. In the *Create A New Vivado Project* dialog box, select **Next**.


* Click the *Browse* button in the project location field of the new project form, navigate to `c:/Xilinx/EEC_EP/labs` and click **Select**. Type **lab3** in the project name field. Make sure the *Create Project Subdirectory* box is checked. Click **Next**.


* Let's create an empty RTL project. We'll specify the sources later. Select the *RTL Project option* in the *Project Type* form and click **Next**. Make sure the *Do not specify sources at this time* box is checked.


* In the *Default Part* form, select the Boards option and choose the **Basys3** board. Click **Next** and then **Finish** to create the Vivado project.

* In the *Sources* panel, select *Add Sources* and follow the necessary steps (**Add or create design sources**) to add the adder circuit provided in the **disp_hex_mux.vhd** file .


* Open the file in Vivado, look at the code. Verify that the code actually implements the block diagram explained above.



### Step 1.2 ### 

In this section, we will check the operation of the circuit in simulation.

The fact that we use an 18-bit counter to generate a low-frequency clock signal makes the simulation of this circuit too long - each iteration takes 218 clock cycles (which doesn't make sense if we just want to check the behavior). The solution is to reduce the size of the counter by changing the value of the constant **N**. 

* Open the source file (**disp_hex_mux.vhd**) and change the line where the constant **N** is defined to:: `constant N: integer :=4;` and then **SAVE**.


* Now add the provided stimuli file (*Add or create simulation sources*) **disp_hex_mux_tb.vhd**. You should have the following file hierarchy in the *Sources* window:

<div>
<img src="lab3/fig3.png" width="800"/>
</div>

* Run the *Run Simulation* process and observe the simulation result. Add the **UUT** internal signals to the waveform window: **qreg** and **sel** and **hex**. Also select the appropriate **Radix** for each signal. You should get something like the figure below. 


<div>
<img src="lab3/fig4.png" width="1000"/>
</div>


* Close the simulation view in the blue bar.

* To avoid code changes between simulation and synthesis, change the **disp_hex_mux** file so that the constant **N** is defined as generic. This way, you will only need to change its value when the module is used in synthesis (*module instantiation*), which is the case in the next step.

### Step 1.3 ### 

In this section, we will check the operation of the circuit on the board, considering that 8 switches define two hexadecimal numbers, which must be displayed on two 7-segment displays. The other two 7-segment displays must show the result of the sum of the two hexadecimal numbers.


* In the *Sources* panel, select *Add Sources* and follow the steps to add the file: **hex_mux_test.vhd**. Open the file in Vivado and observe the code.


* Add the line that maps the generic constant **N** to the value **18**: `generic map (N => 18)`


* Choose the *Open Elaborated Design* operation and open the schematic corresponding to this circuit. It should look something like the one shown in the figure.


<div>
<img src="lab3/fig5.png" width="1000"/>
</div>


* Close the *Elaborated Design* view in the blue bar.


* dd the provided file (*Constraints file*) **basys3_master.xdc**. Make the necessary changes to assign the inputs and outputs of the circuit to the appropriate pins on the Basys3 board. Click **Save**.


* Remember that pin W7 corresponds to segment 6 (sseg[6]) and in that order, pin U7 corresponds to segment 0 (sseg[0]). The display point (pin V7) corresponds to segment 7 (sseg[7]).


<div>
<img src="lab3/fig5a.png" width="600"/>
</div>


* Now all that's left is to run the synthesis, implementation and bitstream generation processes. Select the **hex_mux_test.vhd** file, double-click on *Generate Bitstream* and all the necessary processes will be performed.

* Connect the board to the PC using the USB cable and program the board. Check the circuit is working.

* Close the *Hardware Manager* view in the blue ribbon and turn off the Basys3 using the on-board power switch.

______

## Step 2 - FIFO (Fist-In-First-Out) Register ##

### Step 2.1 ### 

The aim is to implement an elastic FIFO register that allows the interface between two subsystems, as exemplified in the conceptual diagram in the figure.

<div>
<img src="lab3/fig6.png" width="300"/>
</div>


This type of circuit is essential to allow communication between two asynchronous subsystems, that is, ones that do not share the same clock signal (such as the transmission and reception subsystems in a digital communication system). The circuit has two control signals, **wr** and **rd**, to control the writing (wr) and reading (rd) of data. 


In reality, there is no actual reading – the data at the tip of the FIFO is always available for reading. The rd signal simply removes the data from the tip and advances the read pointer to the next position, which becomes the tip.

One way to implement a FIFO is to add a control circuit to a circular register with two pointers. The write pointer points to the head of the register while the read pointer points to the tail. With each write/read operation the corresponding pointer advances one position.

The figure below shows how an 8-bit FIFO works, with read and write requests. This type of register usually also includes indicators for full memory and empty memory, which occur when the read and write pointers are in the same position. To distinguish between the two situations, flip-flops are used, one initialized to '0' and the other to '1'. According to the read and write operations, the value of the flip-flops is updated so that it is always possible to distinguish between the full and empty states.


<div>
<img src="lab3/fig7.png" width="600"/>
</div>



* The VHDL description of a 16-position (4-bit address) FIFO with 8-bit words is provided in the file **fifo.vhd**. Add it to the Vivado project (*Add or create design sources*), open the file, and look at the code.

* To check the operation of the circuit on the board we will use a reduced version of the FIFO, with 3-bit data (4-by-3 FIFO). Add the file **fifo_test.vhd**. Note that the input data is provided by 3 switches, while the **rd** and **wr** commands are generated by pushbuttons. The data read, as well as the full and empty indicators are displayed on 8 LEDs.


* Since pushbuttons can bounce, debounce circuits are used. The VHDL description is provided in the file **debounce.vhd** (we will leave the detailed analysis of this circuit for the next work). Debounce circuits ensure that only one pulse is generated each time the user presses the button. Add this file to the project.


* To synthesize and implement the **fifo_test.vhd** module, you must first right-click on that module (in the Sources window), and then choose the *Set as Top* option that appears in the dialog window. This option allows you to define the module as the circuit to be implemented. 


* The pins file (XDC) should automatically be associated with this new file. Then modify the **basys3_master.xdc** file to match the requirements of this circuit.

* Now all that's left is to run the synthesis, implementation and bitstream generation processes. Double-click on *Generate Bitstream* and all the necessary processes will be carried out.


* Connect the board to the PC using the USB cable and program the board. Check the circuit is working. Write the numbers **1**, **2**, **3** and **4** in the FIFO. Note that the FIFO is full at this point (LED_FULL = ON). Now read in the same order (increase the position of the reading pointer) and note that you will see these values ​​on the LEDs until the FIFO is empty (LED_EMPTY = ON).

* Close the *Hardware Manager* view in the blue ribbon and turn off the Basys3 using the on-board power switch.


______________

## Step 3 - Digital stopwatch ##

### step 3.1 ### 

We will now implement a digital stopwatch. The time is displayed in 3 decimal digits, counting between 00.0 and 99.9 seconds. When it reaches 99.9 it should automatically restart. It should contain a synchronous clear to restart the count, and a **go** signal to suspend/start the count. The circuit is based on a BCD (Binary Coded Decimal) counter, as shown in the figure below. In this format, a number is represented by a sequence of 4-bit BCD digits. For example, the number $139_{10}$ is represented as “0001 0011 1001”.



<div>
<img src="lab3/fig8.png" width="1000"/>
</div>



Since the Basys3 board has a 100MHz oscillator, it is also necessary to implement a Mod-10000000 counter to generate a clock signal with a period of 100 milliseconds (time resolution of the stopwatch). There are several ways to implement this circuit. In the **stop_watch.vhd** file, an implementation based on 3 BCD counters (mod-10 counters) in cascade and a Mod-10000000 counter to generate the 1 millisecond clock signal (ms_tick) is provided. The counters are all controlled by the input clock signal, although they have an enable input that allows defining when they are enabled to increment.


The tenths of a second counter (**d0**) is incremented with the 10Hz clock signal  (`enable<=’1’ when ms_tick=’1’ else ‘0’`). The seconds counter is incremented whenever the tenths of a second counter reaches the end of its count (**d0_reg=9**). Thus, this counter only advances one unit when the previous one reaches the end. Likewise, the tens of seconds counter is only enabled when the previous counters finish counting (**d0_reg=9 and d1_reg=9**).


To test this circuit on the Digilent development board, we will combine this circuit with the code that allows us to use the set of 4 7-segment displays. The (structural) code for the test circuit is provided in the file **stop_watch_test.vhd**. Note that the first digit of the display (hex3) is set to zero and that the **clr** and **go** signals are mapped to pushbuttons.


* Add these two files to your project and analyze the code of each of the files.

* Select the **stop_watch_test.vhd** file and ensure that it is the top file in the *Sources* window .


* Open the file and check if the generic N is well mapped: `generic map (N => 18)`


* Define the circuit pins in the XDC file.

* Follow the necessary procedures to synthesize and implement the circuit on the Digilent board.



_________

## Challenge ##


#### Challenge 1 ####
Simulate the operation of the digital stopwatch from section 1.3. Note that you should reduce the DVSR constant to a lower value to avoid excessive simulation time. To do this, you can create a generic for this constant and map it at the top file level.

#### Challenge 2 ####

Implement a circuit with an 8-position FIFO, with the possibility of viewing the output value (read value) on one of the 7-segment displays. The indication of FIFO full and empty must also be visible on a 7-segment display:
* FIFO full -> segment **a** ON.
* FIFO empty -> segment **d** ON.
